In [1]:
#pip install spotipy --upgrade              # Uncomment this and run it if you haven't installed spotipy before


In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy                          # Set up a config file with ckey and skey. These are available if you go
from spotipy.oauth2 import SpotifyClientCredentials      # to https://developer.spotify.com/, click on Dashboard 
                                        # from the horizontal black menu, login with your normal user info. Click 
# Import Keys                           # on "create an app" if you haven't yet, it doesn't matter what you call it.
from config import clientID, clientSEC           # Then click into
                                        # your project and you should see Client ID and Client Secret. Those are your 
                                        # ckey and skey. 

In [3]:
# Setting up Spotify API info

client_credentials_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSEC)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

1. Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there. 
2. Go to https://spotifycharts.com
2. Choose the country you want to work on. 
3. Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up. 

In [4]:
# Create dataframe from weekly chart data


path = r"input_files/spotify_top200_charts/*.csv"                # The path requires "".csv" at end of file name. 
                                                                 # This is to prevent the program from blowing up
                                                                 # when it hits some kind of hidden file. 

country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1)    # Sets up **main dataframe** with data from **FIRST** file
string = str(glob.glob(path)[0])                                 # in the folder.
year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)   # This line extracts the year and month from the 
                                                                 # **file name**
country_tracks_df[['Year']] = year_month[0]
country_tracks_df[['Month']] = year_month[1]
country_tracks_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1]) 
Tot_Streams1 = country_tracks_df['Streams'].sum()                # Find out total streams in FIRST file in folder.
country_tracks_df[['Stream %']] = country_tracks_df['Streams'] / Tot_Streams1    # New column with % of streams

for file in glob.glob(path)[1:]:                                 # Now that you have the dataframe set up from the 
        temp_df = pd.read_csv(file, header=1)                    # first file in the folder, this iterates through 
        string = str(file)                                       # remaining files
        year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        #print (year_month)
        Tot_Streams2 = temp_df['Streams'].sum()
        temp_df[['Year']] = year_month[0]
        temp_df[['Month']] = year_month[1]
        temp_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1])
        temp_df[['Stream %']] = temp_df['Streams'] / Tot_Streams2
        country_tracks_df = pd.concat([country_tracks_df,        # Adds temperary datafame to end of main dataframe
                                       temp_df])                 # as new rows.  ¿¿¿???? 
        

                                                 
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])  # Sort the new dataframe by year and month




                                                                 # You should get 4,800 rows (24 months x 200 
                                                                 # tracks per month)
country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,yyyy-mm,Stream %
0,1,7 rings,Ariana Grande,5249057,https://open.spotify.com/track/14msK75pk3pA33p...,2019,01,2019-01,0.036990
1,2,Sweet but Psycho,Ava Max,2874281,https://open.spotify.com/track/25sgk305KZfyuqV...,2019,01,2019-01,0.020255
2,3,Wow.,Post Malone,2865682,https://open.spotify.com/track/6MWtB6iiXyIwun0...,2019,01,2019-01,0.020195
3,4,Dancing With A Stranger (with Normani),Sam Smith,2746722,https://open.spotify.com/track/6Qs4SXO9dwPj5GK...,2019,01,2019-01,0.019356
4,5,Giant (with Rag'n'Bone Man),Calvin Harris,2413175,https://open.spotify.com/track/5itOtNx0WxtJmi1...,2019,01,2019-01,0.017006
...,...,...,...,...,...,...,...,...,...
195,196,The Chain - 2004 Remaster,Fleetwood Mac,386369,https://open.spotify.com/track/5e9TFTbltYBg2xT...,2020,12,2020-12,0.002467
196,197,goosebumps,Travis Scott,386084,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,2020,12,2020-12,0.002465
197,198,GO (feat. Juice WRLD),The Kid LAROI,384363,https://open.spotify.com/track/2rPHUAeUPbNgTmK...,2020,12,2020-12,0.002454
198,199,Papi Chulo,Octavian,379760,https://open.spotify.com/track/6Juv5N0dNU1imms...,2020,12,2020-12,0.002425


In [ ]:
# Get Track IDs
track_names = country_tracks_df['Track Name'].to_list()           # Set up list of tracks to iterate through         
track_ids = []                                                    # Empty list to record track IDs into

for track in track_names:                                         # Heads up: with 4800 tracks to process, this takes
    song_results = sp.search(q=track, type='track', limit=1)      # awhile 
    try:
        track_ids.append(song_results['tracks']['items'][0]['id']) # Prevents program from blowing up - few tracks
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   # Just to let you know it's working
    except IndexError:                                             # lack track ids
        track_ids.append(np.nan)                                   # nan if nothing

7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Sweet but Psycho song ID : 7DnAm9FOTWE3cUvso43HhI
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
Dancing With A Stranger (with Normani) song ID : 6Qs4SXO9dwPj5GKvVOv8Ki
Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Without Me song ID : 7lQ8MOhq6IN2w8EYcFNSUk
MIDDLE CHILD song ID : 2JvzF1RMd7lE3KmFlsyZD8
thank u, next song ID : 3e9HZxeyfWwjeyPAMmWSSQ
Don't Call Me Up song ID : 5WHTFyqSii0lmT9R21abT8
Options song ID : 1Q3HtTFGxjmEljAw7Y0CaR
Nothing Breaks Like a Heart (feat. Miley Cyrus) song ID : 27rdGxbavYJeBphck5MZAF
Someone You Loved song ID : 7qEHsqek33rTcFNT9PFqLf
Play song ID : 4DpNNXFMMxQEKl7r0ykkWA
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Going Bad (feat. Drake) song ID : 2IRZnDFmlqMuOrYOLnZZyc
Shallow song ID : 2VxeLyX666F8uXCJ0dZF8B
ZEZE (feat. Travis Scott & Offset) song ID : 0FZ4Dmg8jJJAPJnvBIzD9z
Lost in the Fire song ID : 2vXKRlJBXyOcvZYTdNeckS
R

Desire (with Dimension) song ID : 507UUx311ghfFTMSuHUeyS
changes song ID : 7AFASza1mXqntmGtbxXprO
bellyache song ID : 51NFxnQvaosfDDutk0tams
Fire On Fire - From "Watership Down" song ID : 5rFj8OzffT0iggSjcBaeND
A Million Dreams song ID : 0RoA7ObU6phWpqhlC9zH4Z
Feel It Still song ID : 6QgjcU0zLnzq5OrUoSZ3OK
Jackie Chan song ID : 4kWO6O1BUXcZmaxitpVUwp
Unforgettable song ID : 3B54sVLJ402zGa6Xm4YGNe
Wake Up in the Sky song ID : 0QIjsbm2fh1cJ45XO9eGqq
Under Pressure - Remastered 2011 song ID : 11IzgLRXV7Cgek3tEgGgjw
Havana (feat. Young Thug) song ID : 1rfofaqEpACxVEHIZBJe6W
i apologise if you feel something song ID : 4mrjzz6ETpW3wosJUYmKoL
First Off (feat. Travis Scott) song ID : 3WRIaWsws011vHMd9uzPjG
Budapest song ID : 2ixsaeFioXJmMgkkbd4uj1
Do I Wanna Know? song ID : 5FVd6KXrgO9B3JPmC8OPst
Let Me Down Slowly song ID : 2qxmye6gAegTMjLKEBoR3d
Survival of the Fittest song ID : 7N1Vjtzr1lmmCW9iasQ8YO
Congratulations song ID : 3a1lNhkSLSkpJE4MSHpDu9
Lucky You (feat. Joyner Lucas) song ID : 6

Perfect to Me song ID : 0DC7Zs2qvAbNK1owWQ4mlL
Always Remember Us This Way song ID : 2rbDhOo9Fh61Bbu23T2qCk
Never Enough song ID : 0Gl5s8IhMmQE5YQwM8Qx1J
Desire (with Dimension) song ID : 507UUx311ghfFTMSuHUeyS
If You're Over Me song ID : 01k24g94i1JvkFLQmVEdCd
These Days (feat. Jess Glynne, Macklemore & Dan Caplen) song ID : 5CLGzJsGqhCEECcpnFQA8x
Another One Bites The Dust - Remastered 2011 song ID : 57JVGBtBLCfHw2muk5416J
Praise The Lord (Da Shine) (feat. Skepta) song ID : 7ycWLEP1GsNjVvcjawXz3z
Sincerity Is Scary song ID : 6HguG9HRb1Ke1bhihfE4m8
Money song ID : 2GGMabyHXnJmjY6CXhhB2e
Con Calma song ID : 2EBKmd9f39SGoqAkDoh2Ug
Think About You song ID : 3Kuu5vASpXK8oRsxOvau6P
Pumpy (feat. Deno, Cadet, AJ & Swarmz) song ID : 2r7kFZo4Ua517FKCW39fo9
Taki Taki (with Selena Gomez, Ozuna & Cardi B) song ID : 4w8niZpiMy6qz1mntFA5uM
Needed Time song ID : 1k65BavLispRXs7vzBuRLV
Rise song ID : 69Sy7207dnixZ6w7RSV9Kb
Jocelyn Flores song ID : 7m9OqQk4RVRkw9JJdeAw96
The Other Side song ID : 2cKov

Better song ID : 6gdLyYNuhWBzqwlOFSsXC7
Last Hurrah song ID : 0i0wnv9UoFdZ5MfuFGQzMy
Bally song ID : 0x0oAe2AqvzQmjTtpO5DCd
Nice For What song ID : 3CA9pLiwRIGtUBiMjbZmRw
Bruises song ID : 4Of7rzpRpV1mWRbhp5rAqG
Almost (Sweet Music) song ID : 5Apvsk0suoivI1H8CmBglv
Let You Love Me song ID : 5Fla3zyOCcIRqrDoKLU0DP
Con Calma song ID : 2EBKmd9f39SGoqAkDoh2Ug
R.I.P. (feat. Rita Ora & Anitta) song ID : 6JqpFboOH2bq9pzaYbkKBn
Don't Stop Me Now - Remastered 2011 song ID : 7hQJA50XrCWABAu5v6QZ4i
Fast song ID : 6MjNilJCVKJq0LfFvo5jWJ
Look Back at It song ID : 3Ol2xnObFdKV9pmRD2t9x8
Ruin My Life song ID : 5tAa8Uaqr4VvA3693mbIhU
Youngblood song ID : 2iUXsYOEPhVqEBwsqP70rE
God's Plan song ID : 6DCZcSspjsKoFjzjrWoCdn
BABY song ID : 6epn3r7S14KUqlReYr77hA
Just Got Paid song ID : 0v3BMLcwUmAwyqJaf5997o
I'll Be There song ID : 4r1CoAATVuxag55Ct3Y1aX
Back & Forth song ID : 45wneA2j0Z0WMnYNDO6Ad0
The Greatest Show song ID : 4ylWMuGbMXNDgDd8lErEle
In My Mind song ID : 0E9ZjEAyAwOXZ7wJC0PD33
Hello My Love

Going Bad (feat. Drake) song ID : 2IRZnDFmlqMuOrYOLnZZyc
Grace song ID : 01JMnRUs2YOK6DDpdQASGY
Power is Power song ID : 4cbdPT6uaBOgOQe3fLMofl
Eastside (with Halsey & Khalid) song ID : 0d2iYfpKoM0QCKvcLCkBao
Nobody song ID : 6bTn1ovliI0OkjUNkiMBJq
my strange addiction song ID : 3Tc57t9l2O8FwQZtQOvPXK
Body (feat. brando) song ID : 0BrDfbNc3HiIHPGfTT8DRc
Advice song ID : 2XwcKJW9Q0gaDIZzg0dKQt
you should see me in a crown song ID : 3XF5xLJHOQQRbWya6hBp7d
Earth song ID : 2S1LebN6AXXQqJolBxlWgO
Sixteen song ID : 3pJkBPHadRAxBBo484RipI
Daily Duppy (feat. GRM Daily) song ID : 65VcwXOF7IR5YMXGNhfXQI
Kitchen Kings song ID : 1RJHieKAxYz9qbREvOR5eA
MIDDLE CHILD song ID : 2JvzF1RMd7lE3KmFlsyZD8
lovely song ID : 0u2P5u6lvoDfwTYjAADbn4
Carry On (from the Original Motion Picture "POKÉMON Detective Pikachu") song ID : 3y3brCCecHC3Db18aIOnny
Mr. Brightside song ID : 7oK9VyNzrYvRFo7nQEYkWN
Please Me song ID : 0PG9fbaaHFHfre2gUVo7AN
all the good girls go to hell song ID : 6IRdLKIyS4p7XNiP8r6rsx
Promise

...

The Chain - 2004 Remaster song ID : 5e9TFTbltYBg2xThimr0rU
Be Alright song ID : 3EPXxR3ImUwfayaurPi3cm
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
Do I Wanna Know? song ID : 5FVd6KXrgO9B3JPmC8OPst
No Idea song ID : 7AzlLxHn24DxjgQX73F9fU
Options song ID : 1Q3HtTFGxjmEljAw7Y0CaR
Suicidal song ID : 1iSqfoUFnQwV0QW1EfUit8
Wait a Minute! song ID : 0y60itmpH0aPKsFiGxmtnh
Dancing On My Own song ID : 2BOqDYLOJBiMOXShCV1neZ
OUT WEST (feat. Young Thug) song ID : 6gi6y1xwmVszDWkUqab1qw
I Fall Apart song ID : 75ZvA4QfFiZvzhj2xkaWAh
She Moves In Her Own Way song ID : 2SGBEDwsOAOAHrrdAd304i
Money In The Grave (Drake ft. Rick Ross) song ID : 5ry2OE6R2zPQFDO85XkgRb
Wake Me Up song ID : 0nrRP2bk19rLc0orkWPQk2
Down Like That (feat. Rick Ross, Lil Baby & S-X) song ID : 4etKC4ngPKsQDHUMnYgV3A
P2 song ID : 3kW4z2pIj5VY5fLjBrMRL5
Shape of You song ID : 7qiZfU4dY1lWllzX7mPBI3
Lose You To Love Me song ID : 4l0Mvzj72xxOpRrp6h8nHi
All Girls Are The Same song ID : 4VXIryQMWpIdGgYR4TrjT1
After Hours song ID : 2p8IUWQD

Skechers song ID : 2alc8VZAzDgdAsL2QMk3hu
Piece Of Your Heart song ID : 1DFD5Fotzgn6yYXkYsKiGs
Me & You Together Song song ID : 4AQzk2yNWKs6sYhZi6NzVN
Level of Concern song ID : 6xZ4Q2k2ompmDppyeESIY8
Wonderwall - Remastered song ID : 7ygpwy2qP3NbrxVkHvUhXY
Can't Hold Us - feat. Ray Dalton song ID : 3bidbhpOYeV4knp8AIu8Xn
I Think There's Something You Should Know song ID : 3mRWkQTwau3cl5u2WYE2pt
Mr. Blue Sky song ID : 2RlgNHKcydI9sayD2Df2xp
Jocelyn Flores song ID : 7m9OqQk4RVRkw9JJdeAw96
Lose Somebody song ID : 7xbWAw3LMgRMn4omR5yVn3
WUNNA song ID : 0y7Ao3CVJsyKFmqdjutiIF
Walk Em Down (feat. Roddy Ricch) song ID : 4cSSL3YafYjM3yjgFO1vJg
lovely (with Khalid) song ID : 0u2P5u6lvoDfwTYjAADbn4
One Kiss (with Dua Lipa) song ID : 7ef4DlsgrMEH11cDZd32M6
No Judgement song ID : 0pEiMdUEFUzgFSgIltZ9cw
Dancing In The Moonlight (feat. NEIMY) song ID : 4GSDkZt7GcmzBWmsMcpaVO
Don't Call Me Up song ID : 5WHTFyqSii0lmT9R21abT8
Deep End Freestyle song ID : 2AlYncTpVHKwHb55F9lF6O
Robbery song ID : 6Hj9j

Ride It song ID : 2tnVG71enUj33Ic2nFN6kZ
Dancing In The Moonlight (feat. NEIMY) song ID : 4GSDkZt7GcmzBWmsMcpaVO
Be Kind (with Halsey) song ID : 3Z8FwOEN59mRMxDCtb8N0A
Vossi Bop song ID : 44ZJ4hLD5onikKsVLQaILr
Falling song ID : 1ZMiCix7XSAbfAJlEZWMCp
3 Nights song ID : 1tNJrcVe6gwLEiZCtprs1u
I Don't Care (with Justin Bieber) song ID : 3HVWdVOQ0ZA45FuZGSfvns
Lose Control song ID : 7CHi4DtfK4heMlQaudCuHK
Beautiful People (feat. Khalid) song ID : 4evmHXcjt3bTUHD1cvny97
Higher Love song ID : 6oJ6le65B3SEqPwMRNXWjY
Shotgun song ID : 4ofwffwvvnbSkrMSCKQDaC
everything i wanted song ID : 3ZCTVFBt2Brf31RLEnCkWJ
Strike a Pose (feat. Aitch) song ID : 23GvTfcGK454ppLsts3W44
Ballin' (with Roddy Ricch) song ID : 3QzAOrNlsabgbMwlZt7TAY
Dior song ID : 79s5XnCN4TJKTVMSmOx8Ep
Tommy Lee (feat. Post Malone) song ID : 5QZKcQDp0tCcvMdyqEciBV
Free 22 song ID : 0oS2O93E2VLWwIyigtc8Ci
Señorita song ID : 0TK2YIli7K1leLovkQiNik
Sorry song ID : 7gvd8xj4QgPqbQSsn5pV7d
Can't Hold Us - feat. Ray Dalton song ID : 3b

Party Girl song ID : 5RqR4ZCCKJDcBLIn4sih9l
Rain song ID : 24ySl2hOPGCDcxBxFIqWBu
Flowers (feat. Jaykae and MALIKA) song ID : 5aYWnrsYvJk8NT7PmUcxVN
peace song ID : 7MbT4I8qGntX4fMdqMQgke
WHATS POPPIN (feat. DaBaby, Tory Lanez & Lil Wayne) - Remix song ID : 2MbdDtCv5LUVjYy9RuGTgC
Woi song ID : 69F4G9WD5U1vKrpaCiLznl
Dance Monkey song ID : 2XU0oxnq2qxCpomAAuJY8K
Who's That What's That song ID : 6pRlNmiZ15mI0D0Y9VBjDe
Someone You Loved song ID : 7qEHsqek33rTcFNT9PFqLf
Mood Swings (feat. Lil Tjay) song ID : 5rZlwNFl01HqLWBQGryKSm
hoax song ID : 6MWoRt97mnSTXZhu3ggi9C
Holiday song ID : 6zFMeegAMYQo0mt8rXtrli
Chicago Freestyle (feat. Giveon) song ID : 4wVOKKEHUJxHCFFNUWDn0B
Break My Heart song ID : 017PF4Q3l4DBUiWoXk4OWT
Ego Death (feat. Kanye West, FKA twigs & Skrillex) song ID : 3Q5cLvkKsHJvYltf8k7HkO
This City song ID : 3H3r2nKWa3Yk5gt8xgmsEt
Dior song ID : 79s5XnCN4TJKTVMSmOx8Ep
For The Night (feat. Lil Baby & DaBaby) song ID : 0PvFJmanyNQMseIFrU708S
Dancing In The Moonlight (feat. NEIM

Both song ID : 5tFep7dXGd7vEJ668wTPux
Heather song ID : 4xqrdfXkTW4T0RauPLv3WA
Naive song ID : 2RUhtNBh43RtSg0WBPPq3m
rockstar (feat. 21 Savage) song ID : 0e7ipj03S05BNilyu5bRzt
WAP (feat. Megan Thee Stallion) song ID : 4Oun2ylbjFKMPTiaSbbCih
Head & Heart (feat. MNEK) song ID : 6cx06DFPPHchuUAcTxznu9
Ain't It Different (feat. AJ Tracey & Stormzy) song ID : 6NJfIydBMYrXvlaXsFxmc8
Laugh Now Cry Later (feat. Lil Durk) song ID : 2SAqBLGA283SUiwJ3xOUVI
Lighter (feat. KSI) song ID : 5lWUeSYO61UYgNlFlqcwYl
Mood Swings (feat. Lil Tjay) song ID : 5rZlwNFl01HqLWBQGryKSm
ROCKSTAR (feat. Roddy Ricch) song ID : 7ytR5pFWmSjzHJIeQkgog4
Mood (feat. iann dior) song ID : 4jPy3l0RUwlUI9T5XHBW2m
Savage Love (Laxed - Siren Beat) song ID : 1xQ6trAsedVPCdbtDAmk0c
West Ten song ID : 33EGYJLLwz56YIP5Ex9VKs
GREECE (feat. Drake) song ID : 35RJhm1pEovTBwnNR0zWad
Watermelon Sugar song ID : 6UelLqGlWMcVH1E5c4H7lY
Breaking Me song ID : 3H7ihDc1dqLriiWXwsc2po
Blinding Lights song ID : 0VjIjW4GlUZAMYd2vXMi3b
Go Crazy 

TIP: for this next section, add an "output_files" subfolder to export into. 

In [ ]:
# Add Track IDs to dataframe

country_tracks_df['Track ID'] = track_ids                  # Add new column with track IDs

In [ ]:
# Drop empty songs and export dataframe to csv to back it up

clean_country_tracks_df = country_tracks_df.dropna(how='any')           # Use .dropna() to remove rows with missing data

clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  # Back up to .csv

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")

country_track_ids_df

In [ ]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
tempo = []

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        tempo.append(feat_results[0]['tempo'])
        print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        tempo.append(np.nan)
        

In [ ]:
# Add audio features to dataframe

country_track_ids_df['Danceability'] = danceability     # Add new columns with audio features
country_track_ids_df['Valence'] = valence
country_track_ids_df['Energy'] = energy
country_track_ids_df['Tempo'] = tempo

# Add new columns with product of % and each feature
country_track_ids_df['Danceability_Stream%'] = danceability * country_track_ids_df['Stream %']    
country_track_ids_df['Valence_Stream%'] = energy * country_track_ids_df['Stream %'] 
country_track_ids_df['Energy_Stream%'] = valence * country_track_ids_df['Stream %'] 
country_track_ids_df['Tempo_Stream%'] = tempo * country_track_ids_df['Stream %'] 

In [ ]:
# Back up dataframe again to .csv
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')          # Use .dropna() to remove rows with missing data

clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) #Back up the dataframe to csv again

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

In [ ]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence_Stream%'].sum()

country_tracks_data_groupby

In [ ]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [ ]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH (UK)')         # Set up main title

y_axis = valence_2019['Valence_Stream%']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence_Stream%'], label="2019 Weighted Avg Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence_Stream%'], label="2020 Weighted Avg Valence", marker='o', color='steelblue')  # Plot 2020

plt.xlabel('Months')                                                 # Set up axis titles
plt.ylabel('Valence Score')

plt.xlim(-0.75, len(x_axis)-0.25)                                    # Set up axis limits
plt.ylim(0, 1)

plt.legend()                                                         # Include the legend

plt.show()


In [ ]:
# Compare valence scores with covide infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'United Kingdom']   # Filter for country of your choice

country_covid_df.head()


In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''
country_covid__2020_df.head()

In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby


In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

# No need to add january in Italy
# country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

# country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby


In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 5002100  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('UK: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence_Stream%']
ax1.plot(months, ax1_ydata, label="Weighted Avg Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence_Stream%']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('UK: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence_Stream%")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()